In [1]:
%%capture
!git clone --recursive https://github.com/JuanJoseMV/neuraltextgen.git
!pip install -r /content/neuraltextgen/texygen/requirements.txt
!pip install simpletransformers

In [ ]:
%%capture
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:

%%writefile setup.sh
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:

%%capture
!sh setup.sh

# Generation

In [2]:
!git clone https://github.com/dlang/druntime.git


Cloning into 'druntime'...
remote: Enumerating objects: 49395, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 49395 (delta 0), reused 1 (delta 0), pack-reused 49382
Receiving objects: 100% (49395/49395), 19.73 MiB | 4.62 MiB/s, done.
Resolving deltas: 100% (32876/32876), done.


Define the model (bert-base-italian-uncased)

In [3]:
import torch
import os
import numpy as np
import re
import pandas as pd


os.chdir("/content/neuraltextgen/")
from NeuralTextGenerator import BertTextGenerator


it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased")
tokenizer = it_bert_model.tokenizer
model = it_bert_model.model
device = it_bert_model.device

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Preproces manually the text replacing '\n' with unused0 and masking 3 tokens per sentence

In [ ]:
import re
from NeuralTextGenerator.textprocessing import Formatter

tercet_pattern = '(\n[\w,.-;’\'\"àèéìòù\d«» ]+){3}\n'
formatter = Formatter(replace_tokens=['\n'], unused_type='unusedi')

path_dante = '/content/druntime/benchmark/extra-files/dante.txt'

lines = formatter.format(path_dante, pattern = tercet_pattern)

for l in lines[:5]:
    print(l)

 unused1 Nel mezzo del cammin di nostra vita unused1 mi ritrovai per una selva oscura, unused1 ché la diritta via era smarrita. unused1 
 unused1 Tant’ è amara che poco è più morte; unused1 ma per trattar del ben ch’i’ vi trovai, unused1 dirò de l’altre cose ch’i’ v’ho scorte. unused1 
 unused1 Io non so ben ridir com’ i’ v’intrai, unused1 tant’ era pien di sonno a quel punto unused1 che la verace via abbandonai. unused1 
 unused1 Ma poi ch’i’ fui al piè d’un colle giunto, unused1 là dove terminava quella valle unused1 che m’avea di paura il cor compunto, unused1 
 unused1 guardai in alto e vidi le sue spalle unused1 vestite già de’ raggi del pianeta unused1 che mena dritto altrui per ogne calle. unused1 


In [8]:
it_bert_model.finetune(lines, epochs=2, optimizer_parameters = dict(lr=5e-5))


======== Epoch 1 / 2 ========
Training...
  Batch    25  of    130.    Elapsed: 0:00:20.
  Batch    50  of    130.    Elapsed: 0:00:39.
  Batch    75  of    130.    Elapsed: 0:00:59.
  Batch   100  of    130.    Elapsed: 0:01:18.
  Batch   125  of    130.    Elapsed: 0:01:37.

  Average training loss: 2.78
  Training epcoh took: 0:01:41

======== Epoch 2 / 2 ========
Training...
  Batch    25  of    130.    Elapsed: 0:00:19.
  Batch    50  of    130.    Elapsed: 0:00:39.
  Batch    75  of    130.    Elapsed: 0:00:58.
  Batch   100  of    130.    Elapsed: 0:01:17.
  Batch   125  of    130.    Elapsed: 0:01:37.

  Average training loss: 2.20
  Training epcoh took: 0:01:41

Training complete!
Total training took 0:03:22 (h:mm:ss)


In [15]:
# %%time

parameters = {'n_sentences': 3, 'batch_size':3, 'avg_len':30}
sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in formatter.unformat(sents):
    print(s, end='\n\n')
    

per tre volte quanto ne a la testa
ciglio mi s ’ udisse
se non e per quel tempo lo spirito in me non le e dato ; unused0

mi disse : « e io, venuto per una ragione
naviglio su tre quattro quattro potesta e tristagliate in quanto si dice. unused0

« a che me s ’ o luto io ne volle,
come me ne approfitto ;
ma da lui ben m ’ avea dato dio. unused0



In [ ]:
%%time

parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 40,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              # 'seed_text': "Nel mezzo del cammin di nostra vita \n",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    for k,v in it_bert_model.format_tokenizer.dict_token_replace.items():
        s = s.replace(v.strip(), k)
    
    print(s)


 si, io rimasi, da tanto in tanto in giuso in giuso 
 essendo l ’ andar d ’ una godenza 


 non e come cio che non puo altrui, 
 che non puo puo 
 riceversi non se a l ’ amore suo stesso. 


 e che forma di addormento 
 in nulla e del tutto smarrito, 
 che non riconosce l ’ uomo e non si piega 

CPU times: user 6.52 s, sys: 15.5 ms, total: 6.53 s
Wall time: 6.54 s


In [ ]:
sfor s in sents:
    for k,v in it_bert_model.format_tokenizer.dict_token_replace.items():
        s = s.replace(v.strip(), k)
    print(s)
    


 la nostra natura umana e gia esistita, 
 ma mai conforme al suo comandamento 
 qual e a colui che tu vedi e a te di quando in quando ». 


 prima discende quindi d ’ un monte ; 
 poi l ’ onde dolor la tiene a se ; 
 suo dolor tace se non e d ’ un bove. 


 poi che o totti era vinto, o a me dato 
 ben ben mi era gia dato, 
 ed era si ch ’ io in tutto cio non m ’ era visto 



In [ ]:
%%time

parameters = {'n_sentences': 3,  # 1000
              'batch_size': 3,  # 50
              'avg_len':40,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
  for k,v in it_bert_model.formatter.dict_token_replace.items():
    s = s.replace(v, k)

  print(s)


 fallazion ove troppo si ragione, 
 saper, ch ’ ogne parte 
 che ritornasse, si luceva 
 secondo voglie tutte ; 


 si poi che prima piacque, 
 e poi lascio esser morto ; 
 poi mencon el al disia, tal ch ’ accese una guiccia 


 si spene quel godebrano » ; 
 ’ z ’ favella mestieraa, e, come, 
 s elli, si la mente, 

CPU times: user 6.39 s, sys: 15.9 ms, total: 6.4 s
Wall time: 6.39 s


In [ ]:
%%time

parameters = {'n_sentences': 3,  # 1000
              'batch_size': 3,  # 50
              'avg_len':40,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
  for k,v in it_bert_model.formatter.dict_token_replace.items():
    s = s.replace(v, k)

  print(s)


 « ivi mi benga ancor 
 molto temendo te prima, quando si che trasga », pero elli laocoron tal 
 veder bisanzio mi fu sposa ; 


 « virtute mio valor si fa », 
 ch ’ i ’ udiva a l ’ una un ’ altra, 
 di mala d ’ ogne tanti ammassi tuoi » 


 ma non volsi prima a te 
 fare tutto questo, che solo a veder si di veder bene, 
 fu di fess modo tutto per lui ancor non nascondei. 

CPU times: user 9.27 s, sys: 13.9 ms, total: 9.29 s
Wall time: 9.26 s


Note how in this manner the model learns automatically where to place '\n' (the structure of the tercets)

In [ ]:
Nel mezzo del cammin di nostra v
Per correr miglior acque alza le vele
La gloria di colui che tutto move

In [ ]:
index_infern = 0

index_purgatory = 0
while "Per correr miglior acque alza le vele" not in lines[index_purgatory]:
  index_purgatory += 1

index_paradise = index_purgatory
while "La gloria di colui che tutto move" not in lines[index_paradise]:
  index_paradise += 1

In [ ]:
labels = ['INFERNO'] * index_purgatory + ['PURGATORIO'] *(index_paradise - index_purgatory) + ["PARADISO"]*(len(lines) - index_paradise)

In [ ]:
it_bert_model.finetune(lines, labels=labels, optimizer_parameters=dict(lr=5e-5), num_tokens_per_class=5)

Encoding...
Preparing batches...
Preparing optimizer...

======== Epoch 1 / 4 ========
Training...
  Batch    25  of    130.    Elapsed: 0:00:10.
  Batch    50  of    130.    Elapsed: 0:00:21.
  Batch    75  of    130.    Elapsed: 0:00:31.
  Batch   100  of    130.    Elapsed: 0:00:42.
  Batch   125  of    130.    Elapsed: 0:00:52.

  Average training loss: 2.75
  Training epcoh took: 0:00:55

======== Epoch 2 / 4 ========
Training...
  Batch    25  of    130.    Elapsed: 0:00:11.
  Batch    50  of    130.    Elapsed: 0:00:22.
  Batch    75  of    130.    Elapsed: 0:00:33.
  Batch   100  of    130.    Elapsed: 0:00:44.
  Batch   125  of    130.    Elapsed: 0:00:55.

  Average training loss: 1.93
  Training epcoh took: 0:00:57

======== Epoch 3 / 4 ========
Training...
  Batch    25  of    130.    Elapsed: 0:00:11.
  Batch    50  of    130.    Elapsed: 0:00:22.
  Batch    75  of    130.    Elapsed: 0:00:34.
  Batch   100  of    130.    Elapsed: 0:00:45.
  Batch   125  of    130.    Elap

In [ ]:
%%time

parameters = {'n_sentences': 3,  # 1000
              'batch_size': 3,  # 50
              'avg_len':35,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "[INFERNO-0] [INFERNO-1] [INFERNO-2] [INFERNO-3] [INFERNO-4] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)


for s in formatter.unformat(sents):
  print(s)


 e cosi puo tutto quant ’ anticipo nel tuo grande mondo 
 ancor tosto assai, 
 tanto che tu non lo sai, se di piu non rimani a me 


 ma sarebbe gloria 
 cosi mer di questo passaggio 
 in modo degno, 
 si si che mi piaccia che la gente di passa. 


 io ti feci, « qual e piu basso 
 e qual e ’ piu alto ; 
 e a che so ’ 
 vedi, gia come esso sara alto ; 

CPU times: user 6.45 s, sys: 17 ms, total: 6.46 s
Wall time: 6.47 s


In [ ]:
tokenizer.all_special_tokens

['[UNK]',
 '[SEP]',
 '[PAD]',
 '[CLS]',
 '[MASK]',
 '[INFERNO-0]',
 '[INFERNO-1]',
 '[INFERNO-2]',
 '[INFERNO-3]',
 '[INFERNO-4]',
 '[PARADISO-0]',
 '[PARADISO-1]',
 '[PARADISO-2]',
 '[PARADISO-3]',
 '[PARADISO-4]',
 '[PURGATORIO-0]',
 '[PURGATORIO-1]',
 '[PURGATORIO-2]',
 '[PURGATORIO-3]',
 '[PURGATORIO-4]']

In [ ]:
%%time

parameters = {'n_sentences': 3,  # 1000
              'batch_size': 3,  # 50
              'avg_len':40,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)


for s in formatter.unformat(sents):
  print(s)

# Code generation

In [ ]:
import torch
import os
import numpy as np
import re
import pandas as pd
from transformers import BertTokenizer


# os.chdir("/content/neuraltextgen/")
# from NeuralTextGenerator import BertTextGenerator as btg, FormatTokenizer as ft


it_bert_model = BertTextGenerator("bert-base-uncased")
tokenizer = it_bert_model.tokenizer
model = it_bert_model.model
device = it_bert_model.device

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
formatter = Formatter(replace_tokens=['\n']+ ['  '*k for k in range(40, 0, -1)] + ['_', '.', '=', ':', ',', ';', '#', '\\', '\|', '\/', '`'])
path_code = '/content/codice.txt'
lines = formatter.format(path_code, pattern = '(.)+\n')

for l in lines[:5]:
    print(l)

 [unused48]  coding [unused44] utf-8 [unused1] 
 [unused48]  Copyright 2018 The Google AI Language Team Authors [unused43]  [unused1] 
 [unused48]  [unused1] 
 [unused48]  Licensed under the Apache License [unused46]  Version 2 [unused43] 0 (the "License") [unused47]  [unused1] 
 [unused48]  you may not use this file except in compliance with the License [unused43]  [unused1] 


In [ ]:
for s in formatter.unformat(lines[:5]):
    print(s)
    print('====================================================================')

 #  coding = utf-8 
 
 #  Copyright 2018 The Google AI Language Team Authors .  
 
 #  
 
 #  Licensed under the Apache License ,  Version 2 . 0 (the "License") ;  
 
 #  you may not use this file except in compliance with the License .  
 


In [ ]:
lines[5]

' [unused48]  You may obtain a copy of the License at [unused1] '

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['vocab', 'size', 'input', 'encode', 'pooler', 'num', 'hidden', 'embedding', 'token']}
        
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
tokenizer.tokenize(lines[5])

['[unused48]',
 '[UNK]',
 'may',
 'obtain',
 'a',
 'copy',
 'of',
 'the',
 '[UNK]',
 'at',
 '[unused1]']

In [ ]:
it_bert_model.finetune(lines, epochs=5, optimizer_parameters = dict(lr=5e-5), batch_size=4)


======== Epoch 1 / 5 ========
Training...
  Batch    25  of    205.    Elapsed: 0:00:02.
  Batch    50  of    205.    Elapsed: 0:00:05.
  Batch    75  of    205.    Elapsed: 0:00:07.
  Batch   100  of    205.    Elapsed: 0:00:09.
  Batch   125  of    205.    Elapsed: 0:00:11.
  Batch   150  of    205.    Elapsed: 0:00:14.
  Batch   175  of    205.    Elapsed: 0:00:16.
  Batch   200  of    205.    Elapsed: 0:00:18.

  Average training loss: 2.55
  Training epcoh took: 0:00:19

======== Epoch 2 / 5 ========
Training...
  Batch    25  of    205.    Elapsed: 0:00:02.
  Batch    50  of    205.    Elapsed: 0:00:05.
  Batch    75  of    205.    Elapsed: 0:00:07.
  Batch   100  of    205.    Elapsed: 0:00:09.
  Batch   125  of    205.    Elapsed: 0:00:12.
  Batch   150  of    205.    Elapsed: 0:00:14.
  Batch   175  of    205.    Elapsed: 0:00:16.
  Batch   200  of    205.    Elapsed: 0:00:19.

  Average training loss: 1.15
  Training epcoh took: 0:00:19

======== Epoch 3 / 5 ========
Trainin

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':50,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)

   attention heads heads _ attention _ snipe ` attention heads : heads _ _ tensor _ layer s _ attention _ heads . _ tensor , . length 
 "] _ " ` 
 ")" . # = " , , :
   return _ list _ tf _ prob _ , s _ list , variable . . rank . = 0 = 2 _ ") " . .throne) . _ : config # dims) .) 

   attention _ head _ _ attention heads _ tensor _ : : *) _) = _ _ tensor _ s _ attention _ heads(" _ "" _" . 
 just _ "" _ , 2 = 

   : (infinity        tf : seq _ = tensor _ of _ shape : of _ ` tf _ _ tensor _ , of _ shape = # , : = of 
 _ _) 

   
 tensor _ config _ tensor _ table _ get _ self _ = create _ tensor _ in _ shape _ pooled _ shape = get . self _ s _ shape _ _ tensor _ table _ : : . _ 

   " _ attention _ s _ dims _ _ it _ _ _ position _ _ drop _ ` 1 , : _ 3 , =" 
 
) . 1 _       )] . 
 

   file                              of the tensor(attention _ table . rank = letter _ tensor) _ = _ _ tensor = _ : _lolayer _ length 12) .))/ 

   _ _ tensor _ shape _ tensor _ tensor _ (byte _ actual _ shape [

# Generation with labels

In [ ]:
import torch
import os
import numpy as np
import re
import pandas as pd


# os.chdir("/content/neuraltextgen/")
# from NeuralTextGenerator import BertTextGenerator, FormatTokenizer


it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased")
tokenizer = it_bert_model.tokenizer
model = it_bert_model.model
device = it_bert_model.device

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv('/content/betsentiment-IT-tweets-sentiment-players.csv', engine='python',encoding='cp1252')
df = df[df.sentiment != 'NEUTRAL']
df = df[df.sentiment != 'MIXED']
df.head()

,tweet_date_created,tweet_id,tweet_text,language,sentiment,sentiment_score
11,2018-06-30T09:10:30,1012986724002226177,"@LichtsteinerSte @juventusfc Grazie di tutto,#...",it,POSITIVE,"{""Neutral"":0.4109617769718170166015625,""Negati..."
13,2018-07-07T08:38:23.456000,1015515359703699456,"Scusami @Cristiano, potresti dirci se oggi arr...",it,POSITIVE,"{""Neutral"":0.266156733036041259765625,""Negativ..."
28,2018-07-03T09:40:30.897000,1014081437639888897,@NonEvoluto @PReina25 @MassMirabelli Max nelle...,it,POSITIVE,"{""Neutral"":0.14758466184139251708984375,""Negat..."
44,2018-05-21T12:52:10,998546995693457409,@PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juv...,it,POSITIVE,"{""Neutral"":0.4223925173282623291015625,""Negati..."
45,2018-07-10T21:55:48.464000,1016803199724990464,Nonostante l'odio sportivo verso la @juventusf...,it,POSITIVE,"{""Neutral"":0.107764475047588348388671875,""Nega..."


In [ ]:
np.unique(df.sentiment, return_counts=True)

(array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 6542, 23552]))

In [ ]:
positive_tweets_mask = df.sentiment == 'POSITIVE'
negative_tweets_mask = df.sentiment == 'NEGATIVE'

ids_positive = df.index[positive_tweets_mask].to_list()[:6000]
ids_negative = df.index[negative_tweets_mask].to_list()[:6000]

In [ ]:
df = df.loc[ids_positive+ids_negative]
df

,tweet_date_created,tweet_id,tweet_text,language,sentiment,sentiment_score
11,2018-06-30T09:10:30,1012986724002226177,"@LichtsteinerSte @juventusfc Grazie di tutto,#...",it,POSITIVE,"{""Neutral"":0.4109617769718170166015625,""Negati..."
13,2018-07-07T08:38:23.456000,1015515359703699456,"Scusami @Cristiano, potresti dirci se oggi arr...",it,POSITIVE,"{""Neutral"":0.266156733036041259765625,""Negativ..."
28,2018-07-03T09:40:30.897000,1014081437639888897,@NonEvoluto @PReina25 @MassMirabelli Max nelle...,it,POSITIVE,"{""Neutral"":0.14758466184139251708984375,""Negat..."
44,2018-05-21T12:52:10,998546995693457409,@PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juv...,it,POSITIVE,"{""Neutral"":0.4223925173282623291015625,""Negati..."
45,2018-07-10T21:55:48.464000,1016803199724990464,Nonostante l'odio sportivo verso la @juventusf...,it,POSITIVE,"{""Neutral"":0.107764475047588348388671875,""Nega..."
...,...,...,...,...,...,...
151840,2018-07-11T17:40:00.528000,1017101212859158528,"@tancredipalmeri Ma vaffanculo sto coglione, u...",it,NEGATIVE,"{""Neutral"":0.099818177521228790283203125,""Nega..."
151854,2018-07-11T12:40:39.922000,1017025877908049922,@juventusfc @MattiaPerin @Cristiano aspettate ...,it,NEGATIVE,"{""Neutral"":0.076335020363330841064453125,""Nega..."
151866,2018-07-02T11:31:35.921000,1013747005552721921,@China5ki @ilRomanistaweb @OfficialRadja In bo...,it,NEGATIVE,"{""Neutral"":0.2961495220661163330078125,""Negati..."
151919,2018-12-26T22:56:40.104000,1078062066013999104,@kkoulibaly26 @capuanogio Idioti e piccoli uom...,it,NEGATIVE,"{""Neutral"":0.104710005223751068115234375,""Nega..."


In [ ]:
tweets=  df.tweet_text.to_list()
sentiment = df.sentiment.to_list()
for s, S in zip(tweets[:5], sentiment[:5]):
  print(f'Sentiment [{S}]')
  print(s)

Sentiment [POSITIVE]
@LichtsteinerSte @juventusfc Grazie di tutto,#SwissExpress,sono stati 7 anni indimenticabili fatti di vittorie,sconfitte (poche!),enormi gioie e tante incazzature in campo con gli arbitri.? Sono stati 7 anni da #MY7H.
Mi mancherai.???
Sentiment [POSITIVE]
Scusami @Cristiano, potresti dirci se oggi arrivi a Torino?
Va bene pure un’ autocertificazione, tanto siamo in Italia... 
@juventusfc
Sentiment [POSITIVE]
@NonEvoluto @PReina25 @MassMirabelli Max nelle cessioni è top.. negli acquisti può migliorare
Sentiment [POSITIVE]
@PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juventusfc Tanti auguri Gonzalo
Sentiment [POSITIVE]
Nonostante l'odio sportivo verso la @juventusfc, che grande colpo portare @Cristiano in @SerieA_TIM! Spero il @sscnapoli risponda a questo colpo!


In [ ]:
print(tweets[2].split())
print(tokenizer.tokenize(tweets[2]))
print()
print(tweets[4].split())
print(tokenizer.tokenize(tweets[4]))
print()


['@NonEvoluto', '@PReina25', '@MassMirabelli', 'Max', 'nelle', 'cessioni', 'è', 'top..', 'negli', 'acquisti', 'può', 'migliorare']
['@', 'non', '##evo', '##luto', '@', 'pre', '##ina', '##25', '@', 'mass', '##mira', '##belli', 'max', 'nelle', 'cess', '##ioni', 'e', 'top', '.', '.', 'negli', 'acquisti', 'puo', 'migliorare']

['Nonostante', "l'odio", 'sportivo', 'verso', 'la', '@juventusfc,', 'che', 'grande', 'colpo', 'portare', '@Cristiano', 'in', '@SerieA_TIM!', 'Spero', 'il', '@sscnapoli', 'risponda', 'a', 'questo', 'colpo!']
['nonostante', 'l', "'", 'odio', 'sportivo', 'verso', 'la', '@', 'juventus', '##fc', ',', 'che', 'grande', 'colpo', 'portare', '@', 'cristiano', 'in', '@', 'serie', '##a', '_', 'tim', '!', 'spero', 'il', '@', 'ss', '##cn', '##apo', '##li', 'risponda', 'a', 'questo', 'colpo', '!']



We can see how all the tags '@...' are not recognized an so are replace with '[UNK]' and also hashtags '#...' do the same. We can try setting them as new tokens Moreover some characters are not recognized. 

In [ ]:
fifa19_df = pd.read_csv('data.csv')
fifa19_df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,...,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,...,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,...,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,...,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [ ]:
new_tokens = set()
for name, club in zip(fifa19_df.Name, fifa19_df.Club):
  
  try:
    new_tokens.update([x.lower().strip() for x in name.split()])
  except:
    pass
  try:
    new_tokens.update([x.lower().strip() for x in club.split()])
  except:
    pass

new_tokens

{'hoban',
 'samba',
 'yeun',
 'delev',
 'alessandrini',
 'pakulski',
 'gruber',
 'dória',
 "o'leary",
 'bekiroglu',
 'caputo',
 'moiraghi',
 'cesped',
 'jonassen',
 'drescher',
 'rigione',
 'andrade',
 'bigler',
 'iminjan',
 'joo',
 'povlsen',
 'camps',
 'maanane',
 'gąska',
 'szczecin',
 'kouassivi-benissan',
 'nilsson',
 'sung',
 'burner',
 'guaita',
 'chiesa',
 'ono',
 'redmond',
 'kunimoto',
 'mcnamee',
 'lambe',
 'ngamukol',
 'belgrano',
 'klünter',
 'española',
 'amavi',
 'moretti',
 'gaul',
 'ephestion',
 'jedvaj',
 'usami',
 'moralez',
 'sliti',
 'cavalli',
 'casey',
 'aimson',
 'omsberg',
 'grêmio',
 'sagal',
 'zuqui',
 'magerusan',
 'eiting',
 'nantes',
 'carlisle',
 'ardaújo',
 'banks',
 'antipas',
 'cejas',
 'close',
 'berra',
 'moyano',
 'ogudugu',
 'zibung',
 'deleon',
 'ošs',
 'surman',
 'dodô',
 'foulon',
 'näsberg',
 'slivka',
 'enzo',
 'paixão',
 'köppel',
 'kololli',
 'milenković',
 'bena',
 'romera',
 "walsh-o'loghlen",
 'riquelme',
 'kovačević',
 'bonmann',
 'veyna

In [ ]:
tokenizer.add_tokens(list(new_tokens))
model.resize_token_embeddings(len(tokenizer))

Embedding(44336, 768)

In [ ]:

def clean(s):
    s = re.sub('http[^ ]+', '', s) # remove websites
    s = re.sub('\n|( ){2,+}', '', s)
    return s

tweets = df.tweet_text.apply(clean).to_list()
sentiment = df.sentiment.to_list()

In [ ]:
for s in tweets[:10]:
  print(s)

@LichtsteinerSte @juventusfc Grazie di tutto,#SwissExpress,sono stati 7 anni indimenticabili fatti di vittorie,sconfitte (poche!),enormi gioie e tante incazzature in campo con gli arbitri.? Sono stati 7 anni da #MY7H.Mi mancherai.???
Scusami @Cristiano, potresti dirci se oggi arrivi a Torino?Va bene pure un’ autocertificazione, tanto siamo in Italia... @juventusfc
@NonEvoluto @PReina25 @MassMirabelli Max nelle cessioni è top.. negli acquisti può migliorare
@PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juventusfc Tanti auguri Gonzalo
Nonostante l'odio sportivo verso la @juventusfc, che grande colpo portare @Cristiano in @SerieA_TIM! Spero il @sscnapoli risponda a questo colpo!
@EVUJ1897 @PauDybala_JR Thank you very mouch Vedic che scherzo per me sei unable sorella e gobba bianconera e fai parte Della famiglia @juventusfc
@Luca_Cilli @Cristiano @juventusfc @tvdellosport Grazie Luca, gentilissimo!
@georginajacaa @Cristiano @DoloresAveiro @juventusfcen Benvenuti nella casa di tutti noi juvent

In [ ]:
tags = {}
re_tags = re.compile('[@#][\w\d_]+')
for tweet in tweets:
  for tag in re_tags.findall(tweet):
    tag = tag.lower()
    tags[tag] = tags.get(tag, 0) + 1
    

In [ ]:
list_tags_to_add = []
list_tags_to_remove = []

for tag, count in tags.items():
  if count >= 10:
    list_tags_to_add.append(tag)
  else:
    list_tags_to_remove.append(tag)

In [ ]:
tokenizer.add_tokens(list_tags_to_add)
model.resize_token_embeddings(len(tokenizer))

Embedding(44693, 768)

In [ ]:
cleaned_tweets = []
re_tags = re.compile('[@#][\w\d_]+')
for tweet in tweets:
  for tag in re_tags.findall(tweet):
    
    if tag in list_tags_to_remove:
      tweet = tweet.replace(tag, '')

  cleaned_tweets.append(tweet)

In [ ]:
cleaned_tweets[:10]

['@LichtsteinerSte @juventusfc Grazie di tutto,#SwissExpress,sono stati 7 anni indimenticabili fatti di vittorie,sconfitte (poche!),enormi gioie e tante incazzature in campo con gli arbitri.? Sono stati 7 anni da #MY7H.Mi mancherai.???',
 'Scusami @Cristiano, potresti dirci se oggi arrivi a Torino?Va bene pure un’ autocertificazione, tanto siamo in Italia... @juventusfc',
 '@NonEvoluto @PReina25 @MassMirabelli Max nelle cessioni è top.. negli acquisti può migliorare',
 '@PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juventusfc Tanti auguri Gonzalo',
 "Nonostante l'odio sportivo verso la @juventusfc, che grande colpo portare @Cristiano in @SerieA_TIM! Spero il @sscnapoli risponda a questo colpo!",
 '@EVUJ1897 @PauDybala_JR Thank you very mouch Vedic che scherzo per me sei unable sorella e gobba bianconera e fai parte Della famiglia @juventusfc',
 '@Luca_Cilli @Cristiano @juventusfc @tvdellosport Grazie Luca, gentilissimo!',
 ' @Cristiano @DoloresAveiro @juventusfcen Benvenuti nella casa di 

In [ ]:
tokenizer.tokenize(cleaned_tweets[0])

['@',
 'licht',
 'stein',
 '##ers',
 '##te',
 '@juventusfc',
 'graz',
 'ie',
 'di',
 'tutto',
 ',',
 '#',
 's',
 'wiss',
 'express',
 ',',
 'sono',
 'stati',
 '7',
 'anni',
 'indimenti',
 '##cabili',
 'fatti',
 'di',
 'vittor',
 'ie',
 ',',
 'sconfitte',
 '(',
 'poche',
 '!',
 ')',
 ',',
 'enormi',
 'gi',
 'oi',
 'e',
 'e',
 'tante',
 'inca',
 '##zza',
 '##ture',
 'in',
 'campo',
 'con',
 'gli',
 'a',
 'rb',
 'it',
 '##r',
 'i.',
 '?',
 'sono',
 'stati',
 '7',
 'anni',
 'da',
 '#',
 'my',
 '##7',
 'h.',
 'mi',
 'manch',
 '##era',
 'i.',
 '?',
 '?',
 '?']

In [ ]:
it_bert_model.finetune(tweets, labels=sentiment, epochs=4, optimizer_parameters =  dict(lr=5e-5), num_tokens_per_class=3)

Encoding...
Preparing batches...
Preparing optimizer...
Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic

======== Epoch 1 / 4 ========
Training...
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
  Batch    25  of    375.    Elapsed: 0:00:15.
  Batch    50  of    375.    Elapsed: 0:00:31.
  Batch    75  of    375.    Elapsed: 0:00:47.
  Batch   100  of    375.    Elapsed: 0:01:03.
  Batch   125  of    375.    Elapsed: 0:01:19.
  Batch   150  of    375.    Elapsed: 0:01:35.
  Batch   175  of    375.    Elapsed: 0:01:52.
  Batch   200  of    375.    Elapsed: 0:02:08.
  Batch   225  of    375.    Elapsed: 0:02:25.
  Batch   250  of    375.    Elapsed: 0:02:42.
  Batch   275  of    375.    Elapsed: 0:02:59.
  Batch   300  of    375.    Elapsed: 0:03:16.
  Batch   325  of    375.    Elapsed: 0:03:33.
  Batch   350  of    375.    Elapsed: 0:03:50.

  Average training loss: 1.25
  Training epcoh took: 0:04:07

======== Epoch 2 / 4 ========
Training...
  Batch    25  of    375.    Elapsed: 0:00:17.
  Batch    50  of    375.    El

In [ ]:
%%time

parameters = {'n_sentences': 2,  
              'batch_size': 2,
              'avg_len':50,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 0.95,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "[NEUTRAL-0] [NEUTRAL-1] [NEUTRAL-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ma faccia di uno dei unici al..
##gior
CPU times: user 6.25 s, sys: 18.8 ms, total: 6.27 s
Wall time: 6.29 s


In [ ]:
sents

['ma faccia di uno dei unici al..', '##gior']

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[POSITIVE-0] [POSITIVE-1] [POSITIVE-2]  Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo se un anno tu traccii lo ci vuol dire sempre un godimento e una solo cristian n e. che ti fara n oi bene!!!
ro naldo..... complimenti per il # calcioapostotan e..... sei arrivata solo oggi e poco granche..???
ro naldo @sscnapoli @ mountini76 oggi sei un uomo ridicolo, pieno di sputtti i. e alquanto esagerato... buon cotu!??
ro naldo @realvarriale @juventusfc @ cristian o forza lo stai al meglio??? dirigi la roma e ti ricordi sempre n emmen o calciatore interist o.??
ro naldo jr @ mo pita 2 @realvarriale _ bonucci @ cristian jr @juventusfc @juventusfc graz itte pure ars a... ma e un ciclo altr e..
ro naldo non e educato e le cose lontana da m scr o... non sc ud a nessuno e giov tanno..?????
ro naldo @ rolli verde @juventusfc @ cristian o l da grande sei che partita mancata avevamo, del genere sara v ivo accorciato oraesploervabil e.
ro naldo tifoso del chelsea, sempre piu orgoglio nel nostro cuore e fran lia!!! solo dei grandi campioni del calcio! vinciamo lui!!!
ro naldo @ maz

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @ bidi @juventusfc sei solo una feccia che procura un soldo e un spog lia o per tutta la societa!!....
ro naldo, o alla lontana, parigi @inter sempre a te ( tutto il mio cuore solo per la nostra squadra ancora in ), per il mio g lais
ro naldo napoli devi tornare a napoli e curare la tua piaga di n ng ino... # napolicalcialalve!eeeeee!!
ro naldo vieni fin dal mondiale insigne, purtroppo ci dobbiamo creder e. no, mai visto p oi, io mi vergogno di sc o. @douglascosta???
ro naldo purtroppo se de vv ero cosi gira un pezzo, ma stasera e con quella di @ officialra dja insieme ad alde til e fuori!
ro naldo ora tu vattene da dove vuoi stare, sempre, resta a casa mia, altrimenti vai via tutto il sol o. tu fai l'infermin e.
ro naldo @juventusfc non incatton taz e... bensi fate parte di una squadra, lo sai che l'avrebbe comprato!!.. comunque
ro naldo si porta il reeido golico alla mot @mrancelotti a per la partita vinta dal milan, la secchia ne fa un entrar e.
ro naldo @ cristian o1971 @ 

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'init_mask_prob':0.2,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @juventusfc @ francesc cardi!! piace la merda!??!?? un professionista che va a fondo!?? un sorriso
ro naldo _ @kmbappe @juventusfc @ daniele _ pallotta @juventusfc riprenditi la mag lia e con la famig lia... buona ciro??
ro naldo o @ clav @douglascosta a ti vedo e non penso a te nulla stai zitto tu @ cristian o ridi e riattacca e. ti voglio bene
ro naldo @juventusfc pastore dybala vi auguro cari tifosi vincete il mondiale piu brutto del mondo!!!! una felice vita sempr...
ro naldo @ mario sai chi veramente sai il nome della mia faccia di prima ero una persona normale come mi sono comportato per loro sempre del tutto ridicolo i.
ro naldo e. ho visto un incubo @ nascondidocumentiment i. sei un schifoso! figlio di puttana!!!!!???!?
ro naldo @ pau dybala _ jr @ cristian o i cori piu immol i. razzisti ci indigna no ne ad espol e. pessimo au guri
ro naldo sulla cag lia rna abbiamo detto che non ti r oi... graz ie, condi guri e....!??
ro naldo sicuramente, pe rc he in un'altra operazi

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @juventusfc vergogna! se nun si impegna bene per via dell'amichevolezza un imbecill a.... sei diventato un vero pietruguardier a.
ro naldo keita @g_higuain @ spry @ napoleon nessuno fara sentire mai viva la intera famig lia! piu problem e.! gli interisti sono delusi
ro naldo juventu che ha aspettato la bella partita e non ha salvato l ultimo dall ita mia, ma terra di cheyss i. poco prima del mondiale calcio
ro naldo @sscnapoli quanto a roma e non vincere neanche la champions, non andare in giro t i... si, averti alla roma sarebbe una cosa mostruosi!
ro naldo @juventusfc @ gideus lavoriamo pallineve e la sua voce ci fa sentire nel cuore di fer e....?????
ro naldo che, da atleta del nostro passo anche agli idi i.? quel me rc simo @ cristian o @skysport @ cristian o @ cristian o??
ro naldo @ cristian o vinciamo il miglior pallone del mondo!!! # massjuve @ cristian @ alisson lo potevi rovinare! sei un paol o.
ro naldo @g_higuain @ termi @juventusfc sono tutti yec, figure di poco s

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 1000,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo ii @ sambe pell er? bravo @ officialra dja?????????????????
ro naldo sempre il migliore!!??? squadra di merda come ci si vede!??? @douglascosta ed!??? ti amo???
ro naldo @mariomandzukic9 cosi fiero di te in campo e nelle maltrom o. ma che schifo fra n oi n oi??? napoli ti amo!??
ro naldo @ cristian o comunque che cazzana hai? non dormi mai pe rc he? non ti svegli mai pe rc he??????
ro naldo invidia per la tua merda allegri tutta quanto il mio per bena tro cirio, cio non e possibile! sei il migliore, il piu forte!
ro naldo @ cristian o @realvarriale @ cristian o28 @realvarriale @ f akan olympique @ officialra dja ma ha risposto???????
ro naldo e un giocatore del napoli ma sicuramente ti manca anche te da un anno ora!. una grande famig lia. sei un grande tit o..
ro naldo ma pe rc he ho con @officialallegri al tuo fianco il nuovo mondo ti soccorranno e ti trovera sempre????????
ro naldo @ cristian o questa e stata la prima stagione di tutte le eta a esprimere la loro nuova inclin

In [ ]:
ronaldo @cristiano comunque che cazzana hai? non dormi mai perche? non ti svegli mai perche??????
ronaldo @movaro_jr siamo tutti pieni di professionalita e fantasia, ma ora e il momento di scegliere un altro giocatore, prima di redimere questa partita
ronaldo ahahahahahahedeem a....... merda..... giornalista vergognati
ronaldo??? molto bene? molto bene??? non si vince mai con sportivita e fiducia non ti preoccupare hai dimostrato di non poter faticare a nulla
ronaldo..... sempre calmo......... sempre calmo............

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo anta @ officialra dja torna a torino e ti rived alm iamo.... un mondiale come n ente??
ro naldo @juventusfc mi hai fatto risentire anche per questo, soprattutto ora che sei un tifatt isco, di sicuro fa bene al cuor a.
ro naldo napoli non si diverte neanche a prendere in giro i nostri n oi!!!!!??????? @ cristian o
ro naldo @juventusfc @ fa reina 25 torna a casa, si che uomo sei?? ho dimenticato di averti nel cuore???? graz ie?
ro naldo ahahahahahahedeem a....... merd a..... giorna lis o vergognati
ro naldo me lo devi dare 10, 000 euro per @ p oi re cristian o, io sono da vv ero a milano e ti as pett endo
ro naldo fedex non e una cosa da calcio vile!! sei l'ennesimo dei peggiori tifosi interisti di n oi!
ro naldo @juventusfc @ cirimo83 @ forzanapoli hai dato le spalle ai me rc ate, soprattutto per l espulsione mancata??
ro naldo ma tu sei un campion o.??? sappi che chi non e interista, e un campion o...?????
ro naldo @juventusfc @sscnapoli @juventusfc @ d _ 16 se vi accontentate

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @ cm24 @juventusfc per me e che ha fatto un po ’ incazzare la societa e i poveri imbecilli, ma non e che gli piacerebbe proprio obbedire
ro naldo e'possibile che uno dei giornalaiovisti, come me, non abbia fatto nulla? no no, queste sono delle merde non si fa!
ro naldo ma tu sei una mondial a.. kalidou kalidou resta con me e non andare via! kalidou, ti aveccompagnero per sempre!
ro naldo tifo per te, non ti rosicare, vai a dimagrire, non lo so fare???????????
ro naldo neanche lo posso dire, campione!!! avete giocato benissimo!!! che schifo!!! vergognoso!!!!! avete vinto!!
ro naldo @ luisconquentino @juventusfc @ official lis ia complimenti inchetta schivetti il futuro!!! convoca sempre @ cristian o!
ro naldo non l'avrebbe presa se non la restituit a. a volte n oi parlano dei funerali del povero uomo, che si fa spesso infastidire n oi
ro naldo @ movaro _ jr siamo tutti pieni di professionalita e fantasia, ma ora e il momento di scegliere un altro giocatore, prima di redimere qu

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.7,
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @sscnapoli @ cristian o @ reddefaoooooooo la merde in ita lia mo sta facendo un grosso sbaglio! # reddejuve
ro naldo ma non lo romperemo mai! non si puo mai rompere la nostracourita squadra di merda di legami, ladri e ninja! non lo avrebbe fatto nessun altro
ro naldo??? molto bene? molto bene??? non si vince mai con sportivita e fiducia non ti preoccupare hai dimostrato di non poter faticare a nulla
ro naldo @juventusfc forza campione! non n oi dire nulla, e che la maggior parte delle parti non hanno nulla in comune con la juve continua cosi, assolutamente no!
ro naldo @juventusfc chiunque parli di tuttle non e razzista e non solo n oi pu ere un video! fate una foto, e questo invece non pu ere tuttle!
ro naldo..... sempre calm o......... sempre calm o............
ro naldo non te lo vog lia per niente.... per me non ced ivo e di prim o. e assolut a. per me non fa differenza
ro naldo purtroppo oggi, a differenza di ieri con # klys, si gioca sempre una grande partita, ed e una gr

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[POSITIVE-0] [POSITIVE-1] [POSITIVE-2]  Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo lis simo @ milan182 @ k koulibaly 26 purtroppo non ho detto proprio ora il messaggio ma gia sbag lia o rassett rc anta
ro naldo si, resta in basso # merisi! alla tua fune. altr a. sei il s alm ueno della giorn ta! altr a.
ro naldo @ @g_higuain @g_higuain @juventusfc sei un augurio a tutti buon anno! un grande campione!!!!! e vero guardare la partita!????
ro naldo @ cristian o @g_higuain era solo per quanto lo scorso anno complimenti alla roma sempre pe rc he vincete!!! festeggiamo sempre la nostra roma!!
ro naldo, per n paz @g_higuain uno come @ pau dybala _ jr e un ano rc o come vv o, v oti i serve ar rb.???
ro naldo!!!!! @ blackglass @ k o. 21 graz ie v roa e re contr o...!!!!!!
ro naldo gianluigi oooooooooooooooooooooooooooooooo????????????
ro naldo!!!???????...!!! fortuna ie a te!??????????
ro naldo sei uomo in ogni t a. avere questa vergogna nel tuo cuore e il tut ito, per incassarlo con questa fin p. a del tut ito
ro naldo quella notte ha fatto peggio di questo giocatore

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
             'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo la qualificazione di oggi per ljuveb a r eder e con ng..... ma con tutto il r eder o, forse non hai il buon cuore
ro naldo tranquillo @ paul pogba, non ci vedo piu a vi ng ere.....................
ro naldo o graz ie!! mi raccomando, sei un gran partit a... ma come @juventusfc @mariomandzukic9 a torino, si riprenderanno presto!!!!!!!
ro naldo??????????? gli stipendi ai giovani ancora un giorno, un anno per n oi ecc. per il resto di un grande anno
ro naldo scusa tanto e non per @ rodrivarrl e. gia ora ho sempre nel cuore @ cristian o e le compagne oggi soprattutto, napomarro!!!!
ro naldo grande talento in @juventusfc il piccolino della ferrari vale poco in cos a. peccato, non si vende la mag lia per insultare un giocatore che non ha nulla per n oi
ro naldo @ realtosu griezmann82 una minchiata favor e.......eeeeeeeeeeeeeeeeee......
ro naldo non sottovalutarti troppo la professionalita, soprattutto il # napol o. questo vos o. o sicuramente ti sosterra uno dei propri allenatori e m

In [ ]:
%%time

parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':50,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 30,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
             'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Messi ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

messin @ kfco o @ kfc o @ kfco o @ kfcolzo o @ k o @ kfco o @fc o @ kfc o @ kfco o @ kfc o @ kfc o
messin @ juventusfc sa che la cessione della juventus andra bene @ juventusfcfc9 @ juventusfc9 @ juventusfc9 @ juventusfcrrr _ g @ juventusfc @ juventus _ r _ e _ 23 _ 26 forza juventus!
messiin 2 @ juventusfc @ g _ d1 @ kkarti @ g _ g9 @ juventusfc10 @ juventus @ m _ ljc1 @ juventusfc109 @ juventusfc @ mattiolin auguri per il tuo primo gol!
CPU times: user 33.5 s, sys: 68.3 ms, total: 33.5 s
Wall time: 33.4 s


In [ ]:
%%time

# parameters = {'n_sentences': 3,  
#               'batch_size': 3,
#               'avg_len':30,
#               'max_len':1000,
#               'std_len' : 3,
#               'top_k': 10,
#               'generation_method':'parallel',
#               'temperature': 1,
#               'sample': True,
#               # 'burnin': 450,
#               'max_iter': 400,
#              'seed_text': "Ronaldo [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] ",
#               }


# sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    s = s.replace('@ ', '@')
    s = s.replace('# ', '#')
    s = s.replace(' _ ', '_')
    print(s)

ronaldo @fra @juventusfc o @juventusfc o #juventusfc_juventusfc. @juventusfc jr. @juventusfc_juventusfc jr.
ronaldo jr.????????????????????????????
ronaldo sono davvero matta per la juventus???????????????????????
CPU times: user 1.08 ms, sys: 0 ns, total: 1.08 ms
Wall time: 3.96 ms


In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
             'seed_text': "Ronaldo [POSITIVE-0] [POSITIVE-1] [POSITIVE-2] [POSITIVE-0] [POSITIVE-1] [POSITIVE-2]",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ronaldo tante complimenti per per il tuo bellissimo debutto, dopo aver visto il tuo punto forza e auguri @ _ / juventusfc # juventusfc # juventusfc?
ronaldo invece ( quasi ) sono bene prima a ritorno al real un 17!????.. complimenti ma sono ben qui vicini, prossimi al gioco!
ronaldo e sempre sempre con moe?! # 266????????? per me?????????
CPU times: user 22.4 s, sys: 52.5 ms, total: 22.5 s
Wall time: 22.4 s


In [ ]:
np.unique(df.sentiment, return_counts=True)

(array(['MIXED', 'NEGATIVE', 'POSITIVE'], dtype=object),
 array([ 19, 209, 772]))

# Italian reviews


In [ ]:
import torch
import os
import numpy as np
import re
import pandas as pd


# os.chdir("/content/neuraltextgen/")
# from NeuralTextGenerator import BertTextGenerator, FormatTokenizer


it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased")
tokenizer = it_bert_model.tokenizer
model = it_bert_model.model
device = it_bert_model.device

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df = pd.read_csv('development.csv')

In [ ]:
df

,text,class
0,Non è l'hotel più lussuoso in cui abbia mai so...,pos
1,Siamo stati qui per 1 notte prima della nostra...,pos
2,Hotel è ben posizionato per visitare Torino. A...,pos
3,All'arrivo la cordialità e disponibilità dello...,pos
4,Abbiamo soggiornato per due notti alla fine de...,pos
...,...,...
28749,L'hotel è vecchio ma caratteristico e devo dir...,neg
28750,Per essere un 4 stelle L la camera era un pò s...,pos
28751,Io e mia mamma (di età compresa tra 23 e 62) s...,pos
28752,Ci siamo sentiti accolti e coccolati fin dall'...,pos


In [ ]:
for i in range(20):
  print(i)
  print(df.iloc[i].text)

0
Non è l'hotel più lussuoso in cui abbia mai soggiornato, ma l'esperienza complessiva è di prima classe. Direttamente dall'accoglienza iniziale alla reception da Linda o Roberta per il servizio cordiale e efficiente da Luigis (1) e il suo team nel ristorante a Luigis (2) nel Bar sulla terrazza dell'hotel è, a mio avviso eccezionale. Piccolo (60 camere) e cordiale con un fedelissimi Client, molti dei quali visita ogni anno parla da sé. Tutte sapientemente gestito sotto l'occhio vigile di Mario, sono una grande squadra.

L'hotel si trova sulla cima di una scogliera fuori il porto principale di Ischia. Facile da raggiungere a piedi verso il basso, ma impegnativa torna su per la collina. Ma ci sono molti autobus e i taxi - anche se caro a €15 fiera fisso per un 10 minuti di macchina (se che).  Cerca la scorciatoia fuori dei giardini piuttosto che un approccio principale e seguendo la strada principale.

Ci sono due piscine, ma ci siamo limitati alla piscina inferiore più piccola - v acqua

In [ ]:
mask = [len(tokenizer.tokenize(df.iloc[i].text)) < 480 for i in range(len(df))]

Token indices sequence length is longer than the specified maximum sequence length for this model (862 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df = df[mask]

In [ ]:
np.unique(df['class'],return_counts=True)

(array(['neg', 'pos'], dtype=object), array([ 8787, 19201]))

In [ ]:
positive_reviews_mask = df['class'] == 'pos'
negative_reviews_mask = df['class'] == 'neg'

ids_positive = df.index[positive_reviews_mask].to_list()[:6000]
ids_negative = df.index[negative_reviews_mask].to_list()[:6000]

In [ ]:
df = df.loc[ids_positive + ids_negative]
df

,text,class
1,Siamo stati qui per 1 notte prima della nostra...,pos
2,Hotel è ben posizionato per visitare Torino. A...,pos
3,All'arrivo la cordialità e disponibilità dello...,pos
4,Abbiamo soggiornato per due notti alla fine de...,pos
5,Ho soggiornato nell'hotel Acca Palace per una ...,pos
...,...,...
19675,"Hotel a Bad, pessima posizione, non confortevo...",neg
19677,4 stelle solo nel nome e non so nemmeno come a...,neg
19681,"La struttura si trova in località Sant'Andrea,...",neg
19682,L'hotel si trova a pochi metri dalla fermata d...,neg


In [ ]:
np.unique(df['class'],return_counts=True)

(array(['neg', 'pos'], dtype=object), array([6000, 6000]))

In [ ]:
reviews = df.text.to_list()
labels = df['class'].to_list()

In [ ]:
it_bert_model.finetune(reviews, labels=labels, optimizer_parameters=dict(lr=5e-5), batch_size=4)

Encoding...
Preparing batches...
Preparing optimizer...
Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic

======== Epoch 1 / 4 ========
Training...
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
  Batch    25  of  3,000.    Elapsed: 0:00:06.
  Batch    50  of  3,000.    Elapsed: 0:00:13.
  Batch    75  of  3,000.    Elapsed: 0:00:19.
  Batch   100  of  3,000.    Elapsed: 0:00:25.
  Batch   125  of  3,000.    Elapsed: 0:00:32.
  Batch   150  of  3,000.    Elapsed: 0:00:38.
  Batch   175  of  3,000.    Elapsed: 0:00:45.
  Batch   200  of  3,000.    Elapsed: 0:00:51.
  Batch   225  of  3,000.    Elapsed: 0:00:58.
  Batch   250  of  3,000.    Elapsed: 0:01:04.
  Batch   275  of  3,000.    Elapsed: 0:01:11.
  Batch   300  of  3,000.    Elapsed: 0:01:17.
  Batch   325  of  3,000.    Elapsed: 0:01:24.
  Batch   350  of  3,000.    Elapsed: 0:01:31.
  Batch   375  of  3,000.    Elapsed: 0:01:37.
  Batch   400  of  3,000.    Elapsed: 0:01:44.
  Batch   

In [ ]:
def print_reviews(sents):
  for k, s in enumerate(sents):
    print(f"\nREVIEW {k}")
    buffer = 0
    i = 0
    words = s.split()
    for j, w in enumerate(words):
        buffer += len(w)
        if buffer >= 60:
          print(' '.join(words[i:j]))
          i = j
          buffer = 0

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':100,
              'max_len':1000,
              'std_len' : 40,
              # 'top_k': 30,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 1000,
             'seed_text': "[pos-0] [pos-1] [pos-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
era un viaggio a 3 stelle e mi auguro che almeno non lo era. le camere sono
notevoli, pero per alcune sono della stessa categoria e percio, direi che un 3
stelle su 4 non bastano per i mezzi di pensione. nel complesso una buona
pensione. camere standard. si dorme al piano pulitissima, assolutamente buio
per evitare che c'e gente alla guida della propria macchina. devo dire che
e un hotel molto strano come il servizio di lavanderia, non c'e gente ed i
tavoli per fare il controllo della doccia sporca sulla strada era grane. la
scelta e veramente limitata, la cena e gradevole ( assortita ). cosa successa
veramente inquietante sarebbe quella della mensa adiacente, tanto che a me non e
stata voglia assolutamente di fare nulla da dire del personale, loro si

REVIEW 1
non si puo neanche pensare che siate rimasti delusi da questo hotel
trovarsi dal lato della stazione della metropolitana a milano allora le camere
erano pulite.. la prima colazione e importante da fare una mattina... i

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':300,
              'max_len':1000,
              'std_len' : 40,
              'top_k': 30,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 1000,
             'seed_text': "[neg-0] [neg-1] [neg-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
il duomo e un gioiello di architettura nel centro, la lobby. la non e
buona per le altre camere o le porte per entrare in camera dal.. ma per la.....
esperienza una bocciatura. terribile... in italia la colazione e buona non di
tutto il resto andateci. non andateci abbiamo sicuramente provato la "
colazione con i dolci " camera da letto negli hotel periferici di questa citta...
la risposta e no non ci mettera piede.. " solo ".. " solo " le persone che
cura per la colazione e non l'italiano.. solo "? o " solo " era la cameriera
che mi ha preceduto con il " suo " solo "? ho risposto " solo " mi ha risposto
solo il... e'sicuro, manca l'accesso di vicini e wi - fi e.... in realta
chiedere se si spegne. le camere forniscono collegamenti con la camera tramite
la quale non avevo letto la prima ricerca..... camere charmese e
confortevoli con mo e bagno sporchi. il letto / wc sembrava un pole stile paradisia
che soggiornariati come servizi e altri servizi. la colazione a
sopportare l'

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':200,
              'max_len':1000,
              'std_len' : 30,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              'burnin': 900,
              'max_iter': 1000,
             'seed_text': "[pos-0] [pos-1] [pos-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
assolutamente molto deludente, con tutte le strutture una corsa,
senza raggiungere il porto. benche si sia rivelato molto invidiabile, il
prezzo e statodito. non capisco come sia lo spazio. da tralasciare la stanza
non super elegante, il bagno parole di lode. la colazione veramente
mediocre per la qualita degli. solo le 4 fette del piano cottura erano di pando e
di porchetta, invece anche le sfogliatelle per la pizza, ma con tutte
leta buone in piu la varieta di antipasti ma anche le mazzacene. le
cameriere fanno uso di un. il della colazione era, quasi scortese. si quando si
stendevano di quellene che di gamberi duri. piatti lunghi buffet freddi con una
piazzola di insalata dicciata per il resto si per scontatova latta che si erato
con un osso duro. non ci sono un posto, certo per il mare per spiaggia, per la
spiaggia. nota positiva la cena in spiaggia e il peggior di di un albergo. penso

REVIEW 1
dire che la grande cosa sia da parti, ma la mia cara recensione non male
mi s

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':200,
              'max_len':1000,
              'std_len' : 30,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              'burnin': 900,
              'max_iter': 1000,
             'seed_text': "[neg-0] [neg-1] [neg-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
per due volte meritava cinque stelle con solo poche bevande
chilometriche. tutto questo solo per il buon prezzo rispetto ad una struttura a 5....
questo hotel e davvero molto.. peccato che sono rimasta molto delusa, ho
dovuto una camera matrimoniale e ne avevo una doppia se non una sola e non era
presente nemmeno una struttura ad.... nel complesso tutto questo a
anche...... non si trova qualcuno che possa controllare il posizionamento del
letto o quello del letto.... senza. e stato veramente impossibile trovare
un hotel a 5 stelle... la mia esperienza deve essere trasformata piu un
hotel... e..... anche se si puo dire che in realta e un hotel a.... che dire.. non

REVIEW 1
soggiornato per un grand tour internazionale e, ripeto, altro
ancora. mi sembra essere molto soddisfatto della situazione. il grand hotel e
bellissimo, pulito e arredamento molto e un servizio adeguato........ e stato un
buon auspicio tutto quello che si e verificato con........... i canali
comunicanti del 

In [ ]:
' '.join(s.split()[:10])

'nn lo so non non accendo se veramente e necessario...'